## **E-Commerce Shopping Platform**

*Nithya Cheemalamarri & Leon Tang*

**Project Idea**

Our project idea is to design a shopping platform for E-commerce websites. Specifically, this platform will maintain a shopping cart system where users can add items, keep track of products being sold and their according prices and quanitites, check for specific items in the product inventory, and apply discounts and taxes after the user is finished adding items to their cart.

**Dataset**

To gain data about various products, and their quantities and prices, we decided to use a dataset found on Kaggle that contains sales data collected from a small business that sells women's clothing items. This dataset is a CSV file that has the following columns: InvoiceNo, StockCode, Description, Quantity, InvoiceData, UnitPrice, CustomerID, and Country. For the sake of our program, we kept the Description, Quanity, and UnitPrice columns to have a sample product inventory that a user can choose from. The initial dataset contained 541,910 rows of data, but we shortened it to 500 rows.

The dataset can be found here: https://www.kaggle.com/datasets/shilongzhuang/-women-clothing-ecommerce-sales-data

**Program Description**

This program mimics an E-Commerce platform that is able to read from a dataset containing information about items names, their corresponding prices, and their corresponding quantities.

The **DataProcessor** class is designed for efficient handling and preprocessing of tabular data sourced from a CSV file. Upon initialization, the class reads the CSV file into a *Pandas* DataFrame (self.df). The **preprocess_data** method allows truncation of the DataFrame to a specified length, removal of designated columns, and elimination of duplicate rows. The class also provides methods for extracting dictionaries: **get_price_dictionary** generates a dictionary mapping values from a specified description column to corresponding numeric values in a price column after coercion, while **get_quantity_dictionary** creates a dictionary mapping values from a description column to corresponding values in a quantity column. This class serves as a versatile tool for users to streamline data preprocessing tasks and extract specific information in a dictionary format from tabular datasets.

The **ShoppingCart** class represents an online shopping cart and supports operations like adding items, removing items, calculating the total cost, applying discounts, adding tax, and sorting items by price. The cart utilizes dictionaries to keep track of item quantities and prices.

The program reads input data from external text files (add_items.txt and remove_items.txt) to simulate the addition and removal of items from the shopping cart. Make sure that these files, in addition to the dataset, are all in the same directory. After performing these operations, it calculates and displays the total cost, discounted total, and total with tax. Additionally, it implements the merge sort algorithm to sort items in the cart based on their prices.

**Libraries**

We used the Pandas library on Python to conduct data cleaning on our dataset as well as convert the columns into dictionaries that store information about the products. If running the program on VS Code or any other IDE other than a Jupyter Notebook, install Pandas.

**Expectations & Weaknesses**

We expect that our software successfully responds to the user's commands to add items to the cart if the item exists in the product inventory, or remove items from the cart if the item exists in the cart. Moreover, we expect the cart to display the product's prices, any applied discounts, taxes, and the grand total correctly in ascending order of price.

Currently, this software is limited to specific catalog of products. Future improvements can be made to make it more dynamic and work on various datasets.

In [ ]:
import pandas as pd

In [ ]:
class DataProcessor:
    def __init__(self, file_path):
        self.df = pd.read_csv(file_path)
        self.quantity_dict = {}

    def preprocess_data(self, truncate_end, columns_to_drop):
        self.df = self.df.iloc[:truncate_end]
        self.df = self.df.drop(columns=columns_to_drop, errors='ignore')
        self.df = self.df.drop_duplicates()
        #print(self.df)
        #print (self.df.shape[0])

    def get_price_dictionary(self, description_column, price_column):
        self.df[price_column] = pd.to_numeric(self.df[price_column], errors='coerce')
        price_dict = dict(zip(self.df[description_column], self.df[price_column]))
        return price_dict

    def get_quantity_dictionary(self, description_column, quantity_column):
        quantity_dict = dict(zip(self.df[description_column], self.df[quantity_column]))
        return quantity_dict

In [ ]:
class ShoppingCart:
    def __init__(self, discount_percentage=0, tax_rate=0):
        self.cart = {}
        self.price_log = {}
        self.discount_percentage = discount_percentage
        self.tax_rate = tax_rate

    # add item from cart only if SearchItem is True
    def add_items(self, data_processor, item_to_add, quantity):
        '''Add item to cart only if it exists in the catalog'''
        # Get the price_dict and quantity_dict from DataProcessor
        price_dict = data_processor.get_price_dictionary('Description', 'UnitPrice')
        quantity_dict = data_processor.get_quantity_dictionary('Description', 'Quantity')

        if item_to_add in price_dict and item_to_add in quantity_dict:
            if quantity > 0:
                price = price_dict[item_to_add]
                self.price_log[item_to_add] = price
                self.cart[item_to_add] = quantity
                print(f'{quantity} {item_to_add}(s) added to cart. Price: {price*quantity}')
            else:
                print("Quantity should be a positive number.")
        else:
            print(f'{item_to_add} not found in the catalog. Please try again.')

    def remove_item(self, item, quantity):
        '''remove item from cart only if item is in shopping cart'''
        #check if item in cart
        if item in self.cart:
            #if nunmber of items is greater than number removed
            if self.cart[item] > quantity:
                self.cart[item] -= quantity
                print(f"Removed {quantity} {item}(s) from the cart.")
                print(f"There are {self.cart[item]}(s) remaining in the cart.")
            #if number of items is equal to number removed
            elif self.cart[item] == quantity:
                del self.cart[item]
                del self.price_log[item]
                print(f"Removed all of {item} in the cart.")
            #if number of items is less than number removed
            else:
                print(f"There are only {self.cart[item]} {item}(s) in the cart, \n\
                you asked to remove {quantity}.")
                del self.cart[item]
                del self.price_log[item]
                print(f"We removed all of {item} in the cart.")
        #if item is not in cart
        else:
            print(f"{item} not found in the cart.")

    def calculate_total(self):
        '''calculate total of prices in cart pre-discount and pre-tax'''
        total = 0
        for item in self.cart:
            quantity = self.cart[item]
            price = self.price_log[item]
            total += quantity * price
        return total

    def apply_discount(self):
        total = self.calculate_total()
        discount_amount = (self.discount_percentage / 100) * total
        discounted_total = total - discount_amount
        return discounted_total

    def add_tax(self):
        '''return the total plus the calculated tax amount for the cart'''
        total = self.calculate_total()
        tax_amount = self.tax_rate * total
        total_with_tax = total + tax_amount
        return total_with_tax

    def merge_sort(self, cart):
        if len(cart) > 1:
            mid = len(cart) // 2
            left_half = cart[:mid]
            right_half = cart[mid:]

            self.merge_sort(left_half)
            self.merge_sort(right_half)

            i = j = k = 0

            while i < len(left_half) and j < len(right_half):
                if self.price_log[left_half[i]] < self.price_log[right_half[j]]:
                    cart[k] = left_half[i]
                    i += 1
                else:
                    cart[k] = right_half[j]
                    j += 1
                k += 1

            while i < len(left_half):
                cart[k] = left_half[i]
                i += 1
                k += 1

            while j < len(right_half):
                cart[k] = right_half[j]
                j += 1
                k += 1

    def sort_cart(self):
        '''Implement merge sort algorithm to sort and display items in cart by price'''
        sorted_cart = list(self.cart.keys())
        self.merge_sort(sorted_cart)

        self.cart = {item: self.cart[item] for item in sorted_cart}

        print("\nSorted items in the cart by price:")
        for item, quantity in self.cart.items():
            price = self.price_log[item]
            print(f"{quantity} {item}(s) - Price: {price}")

In [ ]:
def main():
    data_processor = DataProcessor('data-2.csv')
    data_processor.preprocess_data(truncate_end=500, columns_to_drop=['InvoiceNo', 'StockCode', 'InvoiceDate', 'CustomerID', 'Country'])
    data_processor.get_price_dictionary('Description', 'UnitPrice')
    data_processor.get_quantity_dictionary('Description', 'Quantity')

    shopping_cart = ShoppingCart(discount_percentage=10, tax_rate=0.05)

    add_items_file_path = 'add_items.txt'
    with open(add_items_file_path, 'r') as add_items_file:
        for line in add_items_file:
            words = line.strip().split(':')
            item_to_add = words[0]
            quantity_to_add = int(words[1])
            shopping_cart.add_items(data_processor, item_to_add, quantity_to_add)
    print ()
    print("Current items in the cart:")
    for item, quantity in shopping_cart.cart.items():
        print(f"{quantity} {item}(s) - Price: {shopping_cart.price_log[item]}")

    remove_item_file_path = 'remove_items.txt'
    with open(remove_item_file_path, 'r') as remove_items_file:
        for line in remove_items_file:
            words = line.strip().split(':')
            item_to_remove = words[0]
            quantity_to_remove = int(words[1])
            shopping_cart.remove_item(item_to_remove, quantity_to_remove)

    print("\nUpdated items in the cart:")
    for item, quantity in shopping_cart.cart.items():
        print(f"{quantity} {item}(s) - Price: {shopping_cart.price_log[item]}")

    total = round(shopping_cart.calculate_total(), 2)
    discounted_total = round(shopping_cart.apply_discount(), 2)
    total_with_tax = round(shopping_cart.add_tax(), 2)

    print(f"\nTotal: ${total:.2f}")
    print(f"Discounted Total: ${discounted_total:.2f}")
    print(f"Total with Tax: ${total_with_tax:.2f}")

    shopping_cart.sort_cart()

    print("\nFinal items in the cart:")
    for item, quantity in shopping_cart.cart.items():
        print(f"{quantity} {item}(s) - Price: {shopping_cart.price_log[item]}")

if __name__ == "__main__":
    main()

1 CREAM CUPID HEARTS COAT HANGER(s) added to cart. Price: 2.75
1 HOME BUILDING BLOCK WORD(s) added to cart. Price: 5.95
1 HAND WARMER UNION JACK(s) added to cart. Price: 1.85

Current items in the cart:
1 CREAM CUPID HEARTS COAT HANGER(s) - Price: 2.75
1 HOME BUILDING BLOCK WORD(s) - Price: 5.95
1 HAND WARMER UNION JACK(s) - Price: 1.85
Removed all of HAND WARMER UNION JACK in the cart.

Updated items in the cart:
1 CREAM CUPID HEARTS COAT HANGER(s) - Price: 2.75
1 HOME BUILDING BLOCK WORD(s) - Price: 5.95

Total: $8.70
Discounted Total: $7.83
Total with Tax: $9.13

Sorted items in the cart by price:
1 CREAM CUPID HEARTS COAT HANGER(s) - Price: 2.75
1 HOME BUILDING BLOCK WORD(s) - Price: 5.95

Final items in the cart:
1 CREAM CUPID HEARTS COAT HANGER(s) - Price: 2.75
1 HOME BUILDING BLOCK WORD(s) - Price: 5.95
